In [ ]:
import sys
import sklearn as skl
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import os

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

### **1. 데이터 전처리**
   (1) 파생변수 추가
   - 참고 자료


https://dacon.io/competitions/official/235736/codeshare/2877?page=1&dtype=recent

https://dacon.io/codeshare/2882?dtype=recent

In [ ]:
## 변수들을 영문명으로 변경
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'sun', 'mj', 'power']
train.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

#######################################
## 건물별, 요일별, 시간별 발전량 평균 넣어주기
#######################################
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 평균 넣어주기
#######################################
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 표준편차 넣어주기
#######################################
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

### 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('20220601'<=train.date_time)&(train.date_time<'20220602'), 'holiday'] = 1

## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [ ]:
cdhs = np.array([])
for num in range(1,101):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

In [ ]:
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'sun', 'mj', 'power']
train.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

train.drop(['sun', 'mj', 'num_date_time', 'date_time', 'hour'], axis = 1, inplace = True)

(2) 결측치 처리

In [ ]:
train_y = train['power']

# 강수량 결측치 0.0으로 채우기
train['prec'].fillna(0.0, inplace=True)

# 풍속, 습도 결측치 평균으로 채우고 반올림하기
train['wind'].fillna(round(train['wind'].mean(),2), inplace=True)
train['hum'].fillna(round(train['hum'].mean(),2), inplace=True)

In [ ]:
# train set과 동일한 전처리 과정
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind','hum']
test.columns = cols
date = pd.to_datetime(test.date_time)
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.weekofyear
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

## 건물별 일별 시간별 발전량 평균
tqdm.pandas()
test['day_hour_mean'] = test.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.day == x['day']) & (power_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

## 건물별 시간별 발전량 평균 넣어주기
tqdm.pandas()
test['hour_mean'] = test.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_std'] = test.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

test.drop(['hour','date_time', 'num_date_time'], axis = 1, inplace = True)

# pandas 내 선형보간 method 사용
for i in range(100):
    test.iloc[i*168:(i+1)*168, :]  = test.iloc[i*168:(i+1)*168, :].interpolate()


test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

cdhs = np.array([])
for num in range(1,101):
    temp = test[test['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

test = test[['num','temp', 'prec', 'wind', 'hum', 'day', 'month', 'week',
       'day_hour_mean', 'hour_mean', 'hour_std', 'holiday', 'sin_time',
       'cos_time', 'THI', 'CDH']]

In [ ]:
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind','hum']
test.columns = cols
date = pd.to_datetime(test.date_time)
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.weekofyear
test.drop(['num_date_time'], axis = 1, inplace = True)

In [ ]:
train_x = train.drop(columns=['power'])

train_x['wc_temp'] = 13.12 + 0.6215 * train_x['temp'] - 11.37 * (train_x['wind'] ** 0.16) + 0.3965 * train_x['temp'] * (train_x['wind'] ** 0.16)
train_x['heat_index'] = (-8.784695 + 1.61139411 * train_x['temp'] + 2.338549 * train_x['hum'] - 0.14611605 * train_x['temp'] * train_x['hum']
- 0.012308094 * (train_x['temp'] ** 2) - 0.016424828 * (train_x['hum'] ** 2) + 0.002211732 * (train_x['temp'] ** 2) * train_x['hum']
+ 0.00072546 * train_x['temp'] * (train_x['hum'] ** 2) - 0.000003582 * (train_x['temp'] ** 2) * (train_x['hum'] ** 2))

test['wc_temp'] = 13.12 + 0.6215 * test['temp'] - 11.37 * (test['wind'] ** 0.16) + 0.3965 * test['temp'] * (test['wind'] ** 0.16)
test['heat_index'] = (-8.784695 + 1.61139411 * test['temp'] + 2.338549 * test['hum'] - 0.14611605 * test['temp'] * test['hum']
- 0.012308094 * (test['temp'] ** 2) - 0.016424828 * (test['hum'] ** 2) + 0.002211732 * (test['temp'] ** 2) * test['hum']
+ 0.00072546 * test['temp'] * (test['hum'] ** 2) - 0.000003582 * (test['temp'] ** 2) * (test['hum'] ** 2))

   (3) 건물 정보와 병합

In [ ]:
build = pd.read_csv('/content/building_info.csv')
build['태양광용량(kW)'] = build['태양광용량(kW)'].replace('-', 0)
build['ESS저장용량(kWh)'] = build['ESS저장용량(kWh)'].replace('-', 0)
build['PCS용량(kW)'] = build['PCS용량(kW)'].replace('-', 0)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
build['build_type'] = le.fit_transform(build['건물유형'])

In [ ]:
build.columns = ['건물번호', 'build_type', 'space', 'cold_space', 'sun', 'ess', 'pcs']
build['sun'] = build['sun'].astype(float)
build['ess'] = build['ess'].astype(float)
build['pcs'] = build['pcs'].astype(int)

In [ ]:
merged_df = pd.merge(train_x, build, left_on='num', right_on='건물번호', how='inner')
merged_df.drop('건물번호', axis = 1, inplace = True)

merged_test = pd.merge(test, build, left_on='num', right_on='건물번호', how='inner')

   (4) 스케일링

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
scaler.fit(train_x)
transformed_train_X = scaler.transform(train_x)
transformed_test_X = scaler.transform(test)

train = pd.DataFrame(data = transformed_train_X, columns = train_x.columns)
train['power'] = train_y

test = pd.DataFrame(data = transformed_test_X, columns = test.columns)

### 2. **학습**
(1) optuna 활용하여 하이퍼파라미터 적용: 모든 전처리 적용

In [ ]:
!pip install optuna

- 평가 지표 함수 정의

In [ ]:
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_merge_scale.csv')
train_y = train['power']
train.drop(['power'], axis = 1, inplace = True)

- 예측에 사용할 모델 선정하기 위해 pycaret 사용

In [ ]:
!pip install pycaret

In [ ]:
from pycaret import regression
regression.setup(data=train, target=train_y)

In [ ]:
best = regression.compare_models(n_select=1, verbose=False, turbo=True, fold = 10)
best.fit(train, train_y)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 400, 10)
    max_depth = trial.suggest_int('max_depth', 25, 30, 1)
    # min_samples_split = trial.suggest_int('min_samples_split', 2, 6, 1)
    # bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=2,
        max_depth=max_depth,
        bootstrap=False
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = smape(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

In [ ]:
params ={'n_estimators': 390, 'max_depth': 29, 'min_samples_split': 2, 'bootstrap': False}
model = ExtraTreesRegressor(**params)
model.fit(train, train_y)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_merge_scale.csv')

# 테스트 데이터에 대한 예측
preds = model.predict(test)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = preds

(2) autogluon: 전처리 중에서 건물 정보 병합만 안 한 데이터 사용

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/train_final_e.csv')
test_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/test_final_e.csv')

In [ ]:
label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 5

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)

predictor.leaderboard(silent = True)

In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = model_pred

### 3. **앙상블**
- 여러 가지를 앙상블했지만 이렇게 했을 때 성적이 제일 높았습니다.

In [ ]:
opt = pd.read_csv('/content/병합 스케일 + optuna.csv')
aug = pd.read_csv('/content/스케일링 + autogluon_5.csv')
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
sub['answer'] = (opt['answer'] + aug['answer']) / 2